# Random exam questions
This script imports a deposit of questions into a pandas dataframe, then created a random test with a selected number of questions for each lecture. The output is a word document including exam instructions and a text file with correct answers. The script to 

In [1]:
import pandas as pd
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
import random

In [2]:
# Change this number to set how many questions per lecture you want in your questionnaire
Q_nb=2

In [3]:
df=pd.read_excel('Questions.xlsx')
df

,Lecture,Question,Correct answer,Wrong answer 1,Wrong answer 2,Wrong answer 3
0,L1,Is the earth flat?,No,Yes,"Yes, like Mars","Yes, like all other planets in the solar system"
1,L1,Is seawater salty?,Yes,Depends who you ask,No,"It was once, now it's not"
2,L2,Is climate change real?,"Yes, and it is caused by humans","No, it is a hoax","Yes, but it is not caused by humans","No, the Earth is cooling"
3,L2,"On a mountain slope, a landslide goes…",Always downwards,Upwards,It jumps in the sky and then falls down,It depends from the material
4,L3,How high is Everest mountain?,8848m,15000m,3000m,100m
5,L3,How deep is the Mariana trench?,10000m,50m,100m,1000m


In [4]:
Lectures=df['Lecture'].unique()
questionnaire = pd.DataFrame()
for i in Lectures:
 questionnaire=questionnaire.append(df.loc[df['Lecture']==i].sample(n=Q_nb))
questionnaire.reset_index(inplace=True)
questionnaire.drop(columns='index',inplace=True)
questionnaire

,Lecture,Question,Correct answer,Wrong answer 1,Wrong answer 2,Wrong answer 3
0,L1,Is the earth flat?,No,Yes,"Yes, like Mars","Yes, like all other planets in the solar system"
1,L1,Is seawater salty?,Yes,Depends who you ask,No,"It was once, now it's not"
2,L2,Is climate change real?,"Yes, and it is caused by humans","No, it is a hoax","Yes, but it is not caused by humans","No, the Earth is cooling"
3,L2,"On a mountain slope, a landslide goes…",Always downwards,Upwards,It jumps in the sky and then falls down,It depends from the material
4,L3,How high is Everest mountain?,8848m,15000m,3000m,100m
5,L3,How deep is the Mariana trench?,10000m,50m,100m,1000m


In [5]:
#This bit was created thanks to successive iterations with ChatGPT, and modified according to my needs

# create a new Word document
document = Document()

# set font size and style for the document
font_size = Pt(11)
font_style = 'Arial'

# add header to the first page
header = document.sections[0].header
paragraph = header.paragraphs[0]
paragraph.text = 'Name: \nSurname: \nDate:  \n '

# add main body text
document.add_heading('Exam Instructions', level=1)
document.add_paragraph('Instructions:')
document.add_paragraph('1. Write your Name, Surname, and student ID on all sheets')
document.add_paragraph('2. Each question has one or two correct answers')
document.add_paragraph('3. There are no questions without a correct answer')
document.add_paragraph('4. Each question is worth one point, there are no deductions for incorrect answers')
document.add_paragraph('5. If you have any doubts about a question, ask the teacher in the classroom')
document.add_paragraph('6. The written exam is subject to the submission on moodle (or, if agreed with the teacher, via email) of the exercise provided. If such exercise is not present in the submissions, the test will be considered void.')
document.add_paragraph('7. It is not allowed to consult notes, talk to classmates during the exam, or attempt to copy answers from others. Anyone who violates this rule will be asked to leave the room immediately and will have to re-register for the next exam session.')

document.add_page_break()

# create a new text file
text_file = open('answer_keys.txt', 'w')

# loop over the rows of the dataframe
for i, row in questionnaire.iterrows():
    # add the question number and text to the document
    question_number = i + 1
    question_text = f'{question_number}. {row["Question"]}'
    paragraph = document.add_paragraph()
    paragraph.style.font.size = font_size
    paragraph.style.font.name = font_style
    run = paragraph.add_run(question_text)
    run.bold = True

    # shuffle the answers
    answers = [row['Correct answer'], row['Wrong answer 1'], row['Wrong answer 2'], row['Wrong answer 3']]
    random.shuffle(answers)

    # assign the letters A, B, C, D to the shuffled answers
    answer_letters = ['A', 'B', 'C', 'D']
    shuffled_answers = [(answer_letters[i], answer) for i, answer in enumerate(answers)]

    # add the answers to the document
    for letter, answer in shuffled_answers:
        answer_text = f'{letter}. {answer}'
        paragraph = document.add_paragraph(answer_text)
        paragraph.style.font.size = font_size
        paragraph.style.font.name = font_style

    # add the assigned letter of the correct answer to the text file
    correct_answer_index = answers.index(row['Correct answer'])
    correct_answer_letter = answer_letters[correct_answer_index]
    text_file.write(f'{question_number}. {correct_answer_letter}\n')

    # add some spacing between questions
    paragraph = document.add_paragraph('')
    paragraph.style.font.size = font_size
    paragraph.style.font.name = font_style
    paragraph.style.paragraph_format.line_spacing = 0.9


# close the text file
text_file.close()

# save the document
document.save('questionnaire.docx')